## 1.3.6 Data Cleaning

Determine the five most common journals and the total articles for each. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal .

In [208]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('WELLCOME/WELLCOME_APCspend2013_forThinkful.csv', 
                 low_memory=False, encoding='iso-8859-1')


df.head(10)

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88
5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,£2392.20
6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,£2367.95
7,23057412 PMC3495574,ACS,Mol Pharm,Quantitative silencing of EGFP reporter gene b...,£649.33
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,£1294.78


In [209]:
# Renaming last column bc the title is so long...
df.rename(index=str, columns={"COST (£) charged to Wellcome (inc VAT when charged)" : "COST"}, inplace=True)

df.head(3)

,PMID/PMCID,Publisher,Journal title,Article title,COST
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56


In [210]:
# cleaning up the Cost column - getting rid of $ , and .

df['COST'] = df['COST'].apply(lambda x: ''.join(list(filter(str.isdigit, str(x)))))

# careful, bc of decimals... need to rework this a bit

In [211]:
# trying to deal with the decimals

def add_dec(num):
    temp = str(num)
    temp = temp[0:-2] + '.' + temp[-2:]
    temp = float(temp)
    return temp

# and to round up to the next int there's a func - np.rint(number)

df['COST'] = df['COST'].apply(add_dec)

# rounding to the nearest integer bc floats arent reliable with currency
df['COST'] = df['COST'].apply(np.rint)

# now converting back to an int...
df['COST'] = df['COST'].astype(int)

df.head(3)

,PMID/PMCID,Publisher,Journal title,Article title,COST
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,0
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,2381
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",643


In [212]:
df['Journal title'] = df['Journal title'].str.capitalize()
df['Journal title'] = df['Journal title'].str.strip()

mess = list(set(list(df['Journal title'])))

# using this function from https://stackoverflow.com/questions/13252333/python-check-if-all-elements-of-a-list-are-the-same-type
def homogeneous_type(seq):
    iseq = iter(seq)
    first_type = type(next(iseq))
    return first_type if all( (type(x) is first_type) for x in iseq ) else False

#print(homogeneous_type(mess[1:]))
#print(sorted(mess[1:]))

# trying to make a dictionary to correct typos in journal names
# by looking at the journal names in smaller groups
# and then adding them (not in this function)

def starting_letter(letter, outof):
    print('\n***{}***'.format(letter))
    for item in outof:
        # if its a float just get outta there, bc the only float in the list is NaN
        if type(item) != float:
            if item[0] == letter:
                print(item)

# ran the next part to clean up the biggest messes that i could easily find...

#for x in range(ord('A'),ord('Z')+1):
#    starting_letter(chr(x), mess)

In [213]:
correct = {'Americal journal of psychiatry': 'American journal of psychiatry',
          'Acta crystallographica section d,  biological crystallography': 'Acta crystallography d',
           'Acta crystallographica section d: biological crystallography': 'Acta crystallography d',
          'Acta crystallographica, section d': 'Acta crystallography d',
           'Antimicobial agents and chemotherapy':'Antimicrobial agents and chemotherapy',
           'Acta d': 'Acta crystallography d', 'The americal journal of human genetics':'American journal of human genetics',
           'The american journal of human genetics': 'American journal of human genetics',    
           'Angew chems int ed' : 'Angewandte chemie international edition',
           'Am j trop med hyg':'American journal of tropical medicine & hygiene',
           'Acta f': 'Acta crystallographica section f: structural biology and crystallization communications',
            'Biinformatics': 'Bioinformatics online', 
           'Bioinformatics': 'Bioinformatics online', 
           'American journal of bioethics--neuroscience': 'American journal of bioethics',
           'Am j bioeth': 'American journal of bioethics',
           'Haematologica':'Haematologica/the haematology journal',
           'Biinformatics': 'Bioinformatics online','Health & place': 'Health and place',
           'Epigentics': 'Epigenetics', 'Plosone' : 'Plos one' , 'The embo journal': 'Embo',
           'Trop med int health':'Tropical medicine and international health',
           'Journal of general virology': 'Journal of virology',
           'Angewande chemie': 'Angewandte chemie', 'Biol open': 'Biology open',
           'Bmj open': 'Biology open', 'Plos  one': 'Plos one', 'Plos 1': 'Plos one',
           'Plos': 'Plos one',
           'Biochem journal':'Journal of biological chemistry',
           'Biochimie': 'Journal of biological chemistry',
           'Biochemical journals': 'Journal of biological chemistry',
           'Biochemical journal': 'Journal of biological chemistry',
           'Biochemistry': 'Journal of biological chemistry',
           'Biological chemistry' : 'Journal of biological chemistry',
           'Biologicial chemistry': 'Journal of biological chemistry',
           'Health: an interdisciplinary journal for the social study of health, illness and medicine': 'Health'
          }

def correction(value):
    
    # first checking if its a float (that one NaN in there...) so it doesnt throw an error 
    if type(value) != float:
        
        # get rid of all 'the 's because they are inconsistently included in the titles
        if 'the ' in value:
            value = value.replace('the ',' ')
        if 'The ' in value:
            value = value.replace('The ','')
        # also drop the '.'s, inconsistency makes the titles not match
        if '.' in value:
            value = value.replace('.','')
        
        # checking in the dict for the correction
        if value in correct:
            newvalue = correct[value]
            return newvalue
        else:
            return value
    else:
        return value
        
df['Journal title'] = df['Journal title'].apply(correction)



In [214]:
# when the data is cleaned you should be able to just call the top 4 with .head(4) here

top5 = df.groupby('Journal title').count().sort_values(['Article title'], ascending=False).head(5)

top5

,PMID/PMCID,Publisher,Article title,COST
Journal title,,,,
Plos one,201,211,211,211
Journal of biological chemistry,69,71,71,71
Neuroimage,28,29,29,29
Nucleic acids research,25,26,26,26
Plos genetics,23,24,24,24


In [215]:
# Count of total articles per journal, just for the top 5 journals
top5['Article title']

Journal title
Plos one                           211
Journal of biological chemistry     71
Neuroimage                          29
Nucleic acids research              26
Plos genetics                       24
Name: Article title, dtype: int64

In [216]:
#mess2 = list(set(list(df['Journal title'])))
#for x in range(ord('A'),ord('Z')+1):
#    starting_letter(chr(x), mess)

In [217]:
# just double checking that the article titles are unique, so that the count above is correct

df[df['Journal title'] == 'Plos one']

,PMID/PMCID,Publisher,Journal title,Article title,COST
1282,3517619,PLoS,Plos one,HCN1 and HCN2 in Rat DRG Neurons: Levels in No...,1001
1283,3498109,PLoS,Plos one,Fetal alcohol exposure and IQ at age 8: Eviden...,1004
1284,3515553,PLoS,Plos one,Vitamin B-12 status during pregnancy and child...,1011
1285,3522679,PLoS,Plos one,Validation of Dual Energy X-ray Absorptiometry...,1011
1286,3485223,PLoS,Plos one,Associations of different phenotypes of wheezi...,1016
1287,PMC3547059,PLoS,Plos one,"""Involvement of EphB1 receptors signalling in ...",1023
1288,3573029,PLoS,Plos one,Reactive oxygen species modulate the barrier f...,1040
1289,3769269,PLoS,Plos one,Chronic pravastatin but not atorvastatin treat...,1061
1290,3782430,PLoS,Plos one,Expression of HIV-1 Vpu Leads to Loss of the V...,1061
1291,3797097,PLoS,Plos one,Molecular phylogeny of a RING E3 ubiquitin lig...,1061


** Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal .

In [218]:
top5_titles = top5.index.tolist()

def printout(title):
    # make a table to calc from
    table = df.loc[lambda df: df['Journal title'] == title, :'COST']
    # just printing the name to minimize confusion
    print('Journal: {}'.format(title))
    print('Mean cost: {}'.format(table['COST'].mean()))
    print('Median cost: {}'.format(np.median(table['COST'])))
    print('Standard deviation of cost: {}\n'.format(np.std(table['COST'])))
    return

for journal in top5_titles:
    printout(journal)

Journal: Plos one
Mean cost: 39724.71563981043
Median cost: 902.0
Standard deviation of cost: 191084.60441929978

Journal: Journal of biological chemistry
Mean cost: 15575.87323943662
Median cost: 1557.0
Standard deviation of cost: 117661.85924293843

Journal: Neuroimage
Mean cost: 2215.137931034483
Median cost: 2326.0
Standard deviation of cost: 261.9043947884657

Journal: Nucleic acids research
Mean cost: 1149.0
Median cost: 852.0
Standard deviation of cost: 434.33884326552095

Journal: Plos genetics
Mean cost: 84839.375
Median cost: 1718.5
Standard deviation of cost: 275931.04639124556

